# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126296e+02     1.363715e+00
 * time: 0.5358009338378906
     1     1.026921e+01     7.935549e-01
 * time: 1.2791728973388672
     2    -1.257089e+01     8.982430e-01
 * time: 1.3453669548034668
     3    -3.442370e+01     6.689797e-01
 * time: 1.4488768577575684
     4    -4.807416e+01     5.211659e-01
 * time: 1.5514910221099854
     5    -5.714221e+01     1.760893e-01
 * time: 1.6543569564819336
     6    -5.981483e+01     1.244888e-01
 * time: 1.847506046295166
     7    -6.086612e+01     5.670426e-02
 * time: 1.924957036972046
     8    -6.132617e+01     4.420828e-02
 * time: 2.0005269050598145
     9    -6.161837e+01     3.831688e-02
 * time: 2.078310012817383
    10    -6.183810e+01     2.357776e-02
 * time: 2.1548349857330322
    11    -6.199323e+01     1.658755e-02
 * time: 2.681318998336792
    12    -6.206679e+01     1.420210e-02
 * time: 2.7455170154571533
    13    -6.212036e+01     1.349772e-02
 * time: 2.80823802947998

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671064
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666461779